In [1]:
import pandas as pd

import numpy as np

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, r2_score

from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

2025-05-14 08:06:47.386070: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-14 08:06:47.391638: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-14 08:06:47.406643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747220807.423794 3134867 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747220807.429875 3134867 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747220807.448734 3134867 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
def interpolacao_por_ml(df: pd.DataFrame,
                        col_de_treino: list[str],
                        var_de_predicao: str,
                        var_de_pontos: str,
                        n_de_teste: int):

    # Obtendo pontos únicos
    pontos_unicos = df[var_de_pontos].unique()

    # Definição dos modelos
    models = [

        ("ExtraTrees", ExtraTreesRegressor(
            n_estimators=200,
            max_depth=20,
            max_features=2,
            min_samples_split=2,
            min_samples_leaf=1,
            random_state=7
        ))

    ]

    # Defininção de lista de métricas
    metrics = []
    for i in range(len(models)):
        metrics.append([[], []])

    for i in range(n_de_teste):

        # Embaralha os pontos únicos
        np.random.shuffle(pontos_unicos)

        # Dividindo em 70% treino e 30% teste
        split_idx = int(len(pontos_unicos) * 0.8)
        pontos_treino = set(pontos_unicos[:split_idx])
        pontos_teste = set(pontos_unicos[split_idx:])

        # Criando DataFrames de treino e teste
        df_treino = df[df[var_de_pontos].isin(pontos_treino)].copy()
        df_teste = df[df[var_de_pontos].isin(pontos_teste)].copy()

        # Definindo features (X) e variável alvo (y)
        X_train = df_treino[col_de_treino]
        y_train = df_treino[var_de_predicao]

        X_test = df_teste[col_de_treino]
        y_test = df_teste[var_de_predicao]

        # Treinar e avaliar cada modelo
        for j in range(len(models)):
            models[j][1].fit(X_train, y_train)                                 # Treinamento
            y_pred = models[j][1].predict(X_test)                              # Previsão
            metrics[j][0].append(r2_score(y_test, y_pred))                     # R²
            metrics[j][1].append(np.sqrt(mean_squared_error(y_test, y_pred)))  # RMSE

    # Verificando melhor Modelo a partir de r2
    best_model = (0, '', '')

    print('Verificação de Modelos:\n')

    for i in range(len(metrics)):

        r2, rmse = np.mean(metrics[i][0]), np.mean(metrics[i][1])

        if best_model[0] < r2:
            best_model = r2, models[i][0], models[i][1]

        print(f'Modelo: {models[i][0][:3]} \t R²: {r2:.4f} \t RMSE: {rmse:.4f}')

    print(f'\nO melhor modelo de ML para a base de dados é: {best_model[1]}.')

    return best_model[2]

In [3]:
df_aesa = pd.read_csv(f'../datas/interim/3.2.1_aesa_with_idw/aesa_1994-2023_mon_sum_idw.csv')

columns_X = ["lat", "lon", "ano", "mes", "IDW"]

# Escolhendo melhor modelo preditivo
model = interpolacao_por_ml(df_aesa, columns_X, "pr_local", "pnt", 1)

Verificação de Modelos:

Modelo: Ext 	 R²: 0.7420 	 RMSE: 41.2040

O melhor modelo de ML para a base de dados é: ExtraTrees.


In [8]:
def interpolacao_por_cnn(df: pd.DataFrame,
                         col_de_treino: list[str],
                         var_de_predicao: str,
                         var_de_pontos: str,
                         n_de_teste: int):

    # Pontos únicos
    pontos_unicos = df[var_de_pontos].unique()

    # Listas para métricas
    r2_list = []
    rmse_list = []

    for i in range(n_de_teste):

        np.random.shuffle(pontos_unicos)
        split_idx = int(len(pontos_unicos) * 0.8)

        pontos_treino = set(pontos_unicos[:split_idx])
        pontos_teste = set(pontos_unicos[split_idx:])

        df_treino = df[df[var_de_pontos].isin(pontos_treino)].copy()
        df_teste = df[df[var_de_pontos].isin(pontos_teste)].copy()

        # Definindo X e y
        X_train = df_treino[col_de_treino].values
        y_train = df_treino[var_de_predicao].values

        X_test = df_teste[col_de_treino].values
        y_test = df_teste[var_de_predicao].values

        # Redimensionar para 3D: (samples, timesteps=1, features)
        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Limpar sessão anterior (importante em loops com Keras)
        clear_session()

        # Criando modelo CNN
        cnn = Sequential([
            Input(shape=(1, X_train.shape[2])),
            Conv1D(64, kernel_size=3, padding='same', activation='relu'),
            MaxPooling1D(1),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.2),
            Dense(64, activation='relu'),
            Dense(1)
        ])

        cnn.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mse'])

        # Treinamento
        cnn.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=0)

        # Previsão e avaliação
        y_pred = cnn.predict(X_test).flatten()

        r2_list.append(r2_score(y_test, y_pred))
        rmse_list.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    # Resultados finais
    print(f"\nCNN \t Média R²: {np.mean(r2_list):.4f} \t Média RMSE: {np.mean(rmse_list):.4f}")
    return cnn

In [10]:
df_aesa = pd.read_csv(f'../datas/interim/3.2.1_aesa_with_idw/aesa_1994-2023_mon_sum_idw.csv')

columns_X = ["lat", "lon", "ano", "mes", "IDW"]

# Escolhendo melhor modelo preditivo
model = interpolacao_por_cnn(df_aesa, columns_X, "pr_local", "pnt", 10)

552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step
552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 880us/step
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step
552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 976us/step
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step
552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 838us/step

CNN 	 Média R²: 0.7330 	 Média RMSE: 41.9505
